# Convert InfoClinic database structure documentation from PDF to HTML
1. Convert PDF to DOC using www.pdf2doc.com.
2. Copy-paste from Word or LibreOffice Writer to text file.
3. Run this script to build HTML documentation.

<B>PROFIT!<B>

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [10]:
import re
import traceback

INPUT_TXT_FILE_NAME = 'infoclinic-db.txt'
OUTPUT_HTML_FILE_NAME = 'infoclinic-db.html'

In [3]:
f = open(INPUT_TXT_FILE_NAME) 
text = '\n'.join(f.readlines())
#text

In [12]:
# table names and comments
id_regexp_str = '[A-Z]+[A-Z0-9_$]+' # in difference from firebird, we require at least 2 characters in identifier (to prevent false matches)

table_name_regexp = re.compile('\\n{}\\n'.format(id_regexp_str))

tables_text = text.split('\nСтруктура таблицы\n')
table_names = list(map(
    lambda x: re.findall(table_name_regexp, x)[-1].strip(), 
    tables_text[:len(tables_text)-1]
))

table_comments = list(map(
    lambda x: re.split(table_name_regexp, x)[-1].strip(), 
    tables_text[:len(tables_text)-1]
))

# tests
print(table_names[0]=='ACCIDENT')
print(table_names[-1]=='WSCHEMA_DENIED')
print(table_comments[0]=='Случаи заболевания')
print(table_comments[-1]=='Прейскурант Клиники. Несовместимые услуги')

True
True
True
True


In [137]:
header = list(filter(
        lambda x: len(x.strip()) > 0,
        re.split(table_name_regexp, tables_text[0])[0].split('\n')
))

header_html = '''
<h1>{}</h1>
<h2>{}</h2>
<h3>{}</h3>
'''.format(header[0], header[1], '<br>'.join(header[2:]))
header_html = header_html.replace('•', '')

# test
display(HTML(header_html))

In [21]:
# table descriptions
page_number = 1
table_link_regexp = re.compile('\\[({0})\\.({0})\\]'.format(id_regexp_str))
# remove empty lines and page numbers
# [BT_PROFILE_GEN] - generator reference (will not process)
# [DOCTOR.DCODE] - field reference (make link to table)
def process_field_comment(comment, table_link_regexp):
    global page_number
    
    comment = comment.strip()
        
    comment = re.sub(table_link_regexp, '<a href="#\\1">[\\1.\\2]</a>', comment)
    
    if comment.isnumeric() and int(comment) == page_number + 1:
        comment = ''
        page_number += 1
    
    if len(comment) > 0:
        comment += '\n'
        
    return comment


# get table columns description
def get_description(table_str, table_name_regexp, table_link_regexp):
    columns = re.findall(table_name_regexp, table_str)[:-1]
    comments = re.split(table_name_regexp, table_str)[1:-1]
    result = ''
    for i in range(len(columns)):
        comments[i] = ''.join(list(map(
                lambda x: process_field_comment(x, table_link_regexp),
                comments[i].split('\n')
            )))
        result += columns[i].strip() + '\t' + comments[i].strip() + '\n'
    return result
    

table_descriptions = list(map(
    lambda x: get_description(x, table_name_regexp, table_link_regexp), 
    tables_text[1:]
))    

# tests
print(table_descriptions[0])

ACDID	Идентификатор случая заболевания. Генератор [ACCIDENTGEN]
ACDPCODE	Код пациента <a href="#CLIENTS">[CLIENTS.PCODE]</a>
PREDIAG	Диагноза с которым открыт случай. <a href="#DIAGCLIENTS">[DIAGCLIENTS.DGCODE]</a>
PREDCODE	Доктор, который открыл случай. <a href="#DOCTOR">[DOCTOR.DCODE]</a>
PRETREATCODE	Идентификатор приема, в котором был открыт случай. <a href="#TREAT">[TREAT.TREATCODE]</a>
PREDATE	Дата открытия случая
NEEDEXACT	Признак - диагноз требует уточнения, автоматически ставится если код
диагноза заканчивается на .9
EXACTEDDIAG	Уточненный диагноз. Ставится, если диагноз отличается от того, с которым
был открыт случай
Если диагноз менялся несколько раз, то пишется последний
диагноз.<a href="#DIAGCLIENTS">[DIAGCLIENTS.DGCODE]</a>
EXACTEDDCODE	Доктор уточнивший диагноз. <a href="#DOCTOR">[DOCTOR.DCODE]</a>
EXACTEDTREATCODE	Идентификатор приема, в котором последний раз производилось изменение
диагноза случая. <a href="#TREAT">[TREAT.TREATCODE]</a>
EXACTEDDATE	Дата последнего уточ

In [101]:
# build html with description for table
field_name_regexp = re.compile('(?!^ ){}(?=\t)'.format(id_regexp_str))

def build_table_html(table_name, table_comment, table_descr, field_name_regexp):
    
    field_names = re.findall(field_name_regexp, table_descr)
    
    field_descriptions = re.split(field_name_regexp, table_descr)[1:]
    field_descriptions = list(map(lambda x: x.split('\n  \n')[0].strip(), field_descriptions)) 
    field_descriptions = list(map(lambda x: '' if x.isnumeric() else x, field_descriptions))
    
    thead = '''
    <thead>
    <tr>
        <td>Колонка</td>
        <td>Описание</td>
    </tr>
    </thead>
    '''
    html = '<h3><a id="{0}">{0}</a></h3>{1}<p><table>{2}'.format(table_name, table_comment, thead)
    for i in range(len(field_names)):
        html += '\n\t<tr><td>' + field_names[i] + '</td><td>' + field_descriptions[i] + '</td></tr>'
    html += '\n</table><p>'
    return html


# test
table_name = table_names[0]
table_comment = table_comments[0]
table_descr = table_descriptions[0]
html = build_table_html(table_name, table_comment, table_descr, field_name_regexp)

display(HTML(html))

Колонка,Описание
ACDID,Идентификатор случая заболевания. Генератор [ACCIDENTGEN]
ACDPCODE,Код пациента [CLIENTS.PCODE]
PREDIAG,Диагноза с которым открыт случай. [DIAGCLIENTS.DGCODE]
PREDCODE,"Доктор, который открыл случай. [DOCTOR.DCODE]"
PRETREATCODE,"Идентификатор приема, в котором был открыт случай. [TREAT.TREATCODE]"
PREDATE,Дата открытия случая
NEEDEXACT,"Признак - диагноз требует уточнения, автоматически ставится если код диагноза заканчивается на .9"
EXACTEDDIAG,"Уточненный диагноз. Ставится, если диагноз отличается от того, с которым был открыт случай Если диагноз менялся несколько раз, то пишется последний диагноз.[DIAGCLIENTS.DGCODE]"
EXACTEDDCODE,Доктор уточнивший диагноз. [DOCTOR.DCODE]
EXACTEDTREATCODE,"Идентификатор приема, в котором последний раз производилось изменение диагноза случая. [TREAT.TREATCODE]"


In [138]:
table_of_content = '''
<html>

<head>

<style>
table {
    width: 80%;
    border-bottom: 4px solid #dfdfdf; 
    border-radius: 6px; 
    border-collapse: separate;
    border-spacing: 0px;
    margin-left:auto; 
    margin-right:auto;
}
table thead tr td {
    color: #ffffff; 
    font-weight: bold;
    background: #c83240;
    padding: 8px;
}
table tbody tr {background: #F6F6F6;}
table tbody tr:nth-child(2n) {background: #FFFFFF;}
table tbody tr td {border: 1px solid #e8e9eb; padding: 4px}
table tbody tr:hover {background: #ffe8e8; transition-duration: 0.6s;}
</style>
  
</head>

<body>
<center>
''' + header_html + '''
<table>
    <thead>
    <tr>
        <td>Таблица</td>
        <td>Описание</td>
    </tr>
    </thead>
'''

html = ''

try: 
    for i in range(len(table_names)):
        table_name = table_names[i]
        table_comment = table_comments[i]
        table_of_content += '\n\t<tr><td><a href="#{0}">{0}</a></td><td>{1}</td></tr>\n'.format(table_name, table_comment.split('\n')[0]) # only first line of comment in ToC
        #print('<a href=' + file_name + '>' + table_name + '</a>', table_comment, '<br>')
        table_descr = table_descriptions[i]
        html += build_table_html(table_name, table_comment, table_descr, field_name_regexp)
    table_of_content += '\n</table>'
    html += '''
    </center>
    </body>
    </html>'''
    
except Exception as ex:
    print('Exception at line {}:'.format(i))
    print('Table name:', table_name)
    print('Table comment:',table_comment)
    print('Table description:', table_descr)
    print( traceback.format_exc())

#display(HTML(table_of_content))
#display(HTML(table_of_content + html))

f = open(OUTPUT_HTML_FILE_NAME,"w+")
f.write(table_of_content + html)
f.close()

In [ ]:
# git inital setup
!git init
!git config --global user.email "pomka@yandex.ru"
!git config --global user.name "Roman Shekhovtsov"
!git remote add origin git@github.com:rshekhovtsov/infoclinic-db-docs.git

In [34]:
# commit & push
!git add *.ipynb
!git commit -m "- added nicely css; - auto"
!git push origin master

[master f3a9c12] -added foreign keys hrefs; -more reliable page numbers skipping
 1 file changed, 18 insertions(+), 5 deletions(-)
Перечисление объектов: 5, готово.
Подсчет объектов: 100% (5/5), готово.
При сжатии изменений используется до 4 потоков
Сжатие объектов: 100% (2/2), готово.
Запись объектов: 100% (3/3), 742 bytes | 742.00 KiB/s, готово.
Всего 3 (изменения 1), повторно использовано 0 (изменения 0)
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
To github.com:rshekhovtsov/infoclinic-db-docs.git
   5d0102c..f3a9c12  master -> master
